In [ ]:
import torch
import torch.nn as nn
import torchvision.transforms as T
from PIL import Image
import matplotlib.pyplot as plt
import os

# Set device
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
ENCODER_PATH = "model_checkpoints/checkpoints/recolored_semantic_layouts/latent4_lossL1_base32_img1024/checkpoints/encoder_final.pt"
DECODER_PATH = "model_checkpoints/checkpoints/recolored_semantic_layouts/latent4_lossL1_base32_img1024/checkpoints/decoder_final.pt"


In [ ]:

from src.autoencoder import AutoEncoderFactory

# === Step 1: Build encoder/decoder from factory ===
factory = AutoEncoderFactory(
    name="latent4_lossL1_base32_img1024",
    in_channels=3,
    out_channels=3,
    base_channels=32,          # ✅ Matches checkpoint
    latent_channels=4,
    depth=3,
    channel_multiplier=2
)
encoder, decoder = factory.build()
encoder, decoder = encoder.to(device), decoder.to(device)

# === Step 2: Confirm architecture ===
print(">>> First conv shape:", next(encoder.parameters()).shape)  # Should be [64, 3, 4, 4]

# === Step 3: Load pretrained weights ===
encoder.load(ENCODER_PATH, map_location=device)
decoder.load(DECODER_PATH, map_location=device)

# === Step 4: Set to evaluation mode ===
encoder.eval()
decoder.eval()

# ✅ Ready to use




In [ ]:
from PIL import Image
import torchvision.transforms as T
import matplotlib.pyplot as plt
import torch

# === Step 1: Load image ===
image_path = "data/recolored_images/0abe65fc-108e-4f9b-a8ff-29758234b9ec_image.png"  # Update this
img = Image.open(image_path).convert("RGB")

# === Step 2: Preprocess to tensor ===
transform = T.Compose([
    T.Resize((1024, 1024)),
    T.ToTensor(),                 # Converts to [0,1]
    T.Normalize([0.5]*3, [0.5]*3) # Scales to [-1,1]
])
x = transform(img).unsqueeze(0).to(device)  # [1, 3, 1024, 1024]

# === Step 3: Encode to latent ===
with torch.no_grad():
    z = encoder(x)  # [1, latent_channels, H', W']

# === Step 4: Decode back to image ===
with torch.no_grad():
    x_recon = decoder(z)  # [1, 3, 1024, 1024]

# === Step 5: Convert to displayable image ===
postprocess = T.Compose([
    T.Normalize([-1] * 3, [2] * 3),     # Rescale [-1, 1] -> [0, 1]
    T.ToPILImage()
])
recon_img = postprocess(x_recon.squeeze().cpu())

# === Step 6: Visualize ===
fig, axes = plt.subplots(1, 2, figsize=(12, 6))
axes[0].imshow(img)
axes[0].set_title("Original")
axes[1].imshow(recon_img)
axes[1].set_title("Reconstructed")
for ax in axes:
    ax.axis("off")
plt.tight_layout()
plt.show()


In [ ]:
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import DataLoader
from torchvision import transforms as T
from torchvision.datasets import ImageFolder


device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

image_dir = "logs/images"
checkpoint_dir = "logs/checkpoints"
dataset_dir = "data/recolored_images"

name = "test_123"
factory = AutoEncoderFactory(
    name=name,
    in_channels=3,
    out_channels=3,
    base_channels=32,
    latent_channels=4,
    depth=3,
    channel_multiplier=2
)


encoder, decoder = factory.build()
encoder.to(device)
decoder.to(device)
optimizer = optim.Adam(list(encoder.parameters()) + list(decoder.parameters()), lr=1e-4)
loss_fn = nn.L1Loss()


In [ ]:
from torch.utils.data import DataLoader
import torchvision.transforms as T
from torch.utils.data import Dataset

class FlatImageDataset(Dataset):
    def __init__(self, root_dir, transform=None):
        self.root_dir = root_dir
        self.transform = transform
        self.image_paths = [
            os.path.join(root_dir, fname)
            for fname in os.listdir(root_dir)
            if fname.lower().endswith(('.png', '.jpg', '.jpeg'))
        ]

    def __len__(self):
        return len(self.image_paths)

    def __getitem__(self, idx):
        img_path = self.image_paths[idx]
        img = Image.open(img_path).convert("RGB")
        if self.transform:
            img = self.transform(img)
        return img


transform = T.Compose([
    T.Resize((1024, 1024)),
    T.ToTensor(),
    T.Normalize([0.5] * 3, [0.5] * 3)
])

dataset = FlatImageDataset(root_dir="data/recolored_images", transform=transform)
dataloader = DataLoader(dataset, batch_size=4, shuffle=True)



In [ ]:
from modules.utils import Logger



In [ ]:
from modules.autoencoder import (
    AutoEncoderTrainer,
    AutoEncoderFactory
)

trainer = AutoEncoderTrainer(
    encoder=encoder,
    decoder=decoder,
    dataloader=dataloader,
    optimizer=optimizer,
    loss_fn=loss_fn,
    device=device,
    image_dir=image_dir,
    checkpoint_dir=checkpoint_dir,
    checkpoint_interval=2
)

trainer.train(epochs=3, log_images_every=2)


In [ ]:
import torch
from modules.autoencoder import AutoEncoderFactory
from modules.utils import Logger

# === Configuration ===
DEVICE = "cuda" if torch.cuda.is_available() else "cpu"
ENCODER_PATH = "path/to/nonexistent/encoder_weights.pt" # Path that will cause an error
DECODER_PATH = "path/to/your/decoder_weights.pt"
LOG_FILE = "model_setup.log"

# Initialize logger to None before the try block
logger = None 
try:
    # === Step 1: Initialize Logger ===
    logger = Logger(info=True, debug=True, log_file=LOG_FILE, source="ModelLoader")
    logger.info(f"Using device: {DEVICE}")

    # === Step 2: Build encoder/decoder from factory ===
    factory = AutoEncoderFactory(
        name="latent4_lossL1_base32_img1024",
        base_channels=32,
        logger=logger
    )
    encoder, decoder = factory.build()
    encoder, decoder = encoder.to(DEVICE), decoder.to(DEVICE)
    
    # === Step 3: Load pretrained weights ===
    logger.info("Loading pretrained weights...")
    # This line will fail and trigger the except block
    encoder.load(ENCODER_PATH, map_location=DEVICE) 
    decoder.load(DECODER_PATH, map_location=DEVICE)
    
    logger.info("✅ Models are ready to use.")

except FileNotFoundError:
    logger.error(f"FATAL: Weight file not found. Checked path: {ENCODER_PATH}")
except Exception as e:
    logger.error(f"An unexpected error occurred during model setup: {e}")

finally:
    # This block will ALWAYS run, ensuring the log file is closed.
    if logger:
        logger.info("Script finished. Closing log.")
        logger.close()

In [1]:
import torch
from layout_generator.models.diffusion import DiffusionModel
from layout_generator.utils.logger import Logger
from layout_generator.utils.factories import DiffusionFactory

# === Configuration ===
DEVICE = "cuda" if torch.cuda.is_available() else "cpu"
LOG_FILE = "diffusion_factory_test.log"

# Initialize logger to None before the try block
logger = None
try:
    # === 1. Initialize Logger ===
    # All factory operations will be captured in the console and the log file.
    logger = Logger(info=True, debug=True, log_file=LOG_FILE, source="FactoryTest")
    logger.info(f"Starting diffusion factory test on device: {DEVICE}")

    # === 2. Initialize Diffusion Factory ===
    # We pass the logger to the factory. It will announce its initialization.
    diffusion_factory = DiffusionFactory(
        name="diffusion_test_v1",
        in_channels=4,          # Latent channels from autoencoder
        out_channels=4,         # Latent channels from autoencoder
        base_channels=128,
        depth=3,
        cond_dim=256,           # Dimension of text/layout embedding
        time_dim=256,
        schedule_type="cosine", # Using a cosine noise schedule
        timesteps=1000,
        logger=logger           # Pass the logger instance here
    )

    # === 3. Build the Diffusion Model ===
    # The factory will log the creation of the UNet and NoiseScheduler.
    diffusion_model = diffusion_factory.build(device=DEVICE)

    # === 4. Print Model Info ===
    # Log the information from the successfully created model.
    model_info = diffusion_model.info()
    logger.info("--- Diffusion Model Details ---")
    logger.info(f"Model Name: {model_info.get('name')}")
    logger.info(f"UNet Parameters: {model_info.get('unet_params'):,}")
    logger.info(f"Scheduler Type: {model_info.get('scheduler_type')}")
    logger.info("-----------------------------")
    logger.info("✅ Factory test completed successfully.")

except Exception as e:
    if logger:
        logger.error(f"An error occurred during the factory test: {e}")
    else:
        print(f"An error occurred before logger was initialized: {e}")

finally:
    # This block ALWAYS runs, ensuring the log file is properly closed.
    if logger:
        logger.close()

[INFO] [FactoryTest] [2025-08-17 17:05:12] Starting diffusion factory test on device: cpu
[INFO] [FactoryTest] [2025-08-17 17:05:12] DiffusionFactory initialized for model 'diffusion_test_v1'
[DEBUG] [FactoryTest] [2025-08-17 17:05:12] Config: {
    "name": "diffusion_test_v1",
    "in_channels": 4,
    "out_channels": 4,
    "base_channels": 128,
    "depth": 3,
    "cond_dim": 256,
    "time_dim": 256,
    "group_norm_groups": 8,
    "time_embedding_scale": 10000.0,
    "schedule_type": "cosine",
    "timesteps": 1000,
    "beta_start": 0.0001,
    "beta_end": 0.02
}
[INFO] [FactoryTest] [2025-08-17 17:05:12] Building Diffusion model components...
[DEBUG] [FactoryTest] [2025-08-17 17:05:12] UNet params: 12,256,388
[DEBUG] [FactoryTest] [2025-08-17 17:05:12] Scheduler: cosine with 1000 timesteps
[INFO] [FactoryTest] [2025-08-17 17:05:12] ✓ DiffusionModel built successfully
[INFO] [FactoryTest] [2025-08-17 17:05:12] --- Diffusion Model Details ---
[INFO] [FactoryTest] [2025-08-17 17:05